In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}


<IPython.core.display.Javascript object>

In [2]:
%matplotlib inline
import ipywidgets
from funciones import *
from ipywidgets import fixed
from functools import partial
from IPython.display import display
from mpl_toolkits.mplot3d import axes3d

# backends: notebook, inline, ipympl...


In [3]:

puntos_prueba1 = [1.5,2.5,3.5]
puntos_prueba2 = [0.4,0.6,0.8]

'''
resultados = rutina_doble(num_bandas=8,
           archivo_control=ctl_num([20,512,50,500], 'control-2mat.ctl'),
           control=[2.35,1.0,0.81,0.19], #e1,e2,e3,a1,a2,a3
           puntos1=[1.5,2.5,3.5],
           puntos2=[0.4,0.6,0.8],
           tipo=['2mat',0,2], # num, mat o lineal. indice (en control) para introduccion   
           cifras=2, # solo se usa para el balance de proporciones
           nombre='rutina-doble',
           funcion=partial(balance_proporcional2,
                           puntos=[0.4,0.6,0.8],
                           material=1,
                           cifras=2
                          )
          )
'''

with open('resultados-rutina-doble.pkl', 'rb') as f:
    resultados0 = pickle.load(f)

print(resultados0['lista'])
    

[[1.5, 0.4], [2.5, 0.4], [3.5, 0.4], [1.5, 0.6], [2.5, 0.6], [3.5, 0.6], [1.5, 0.8], [2.5, 0.8], [3.5, 0.8]]


In [4]:
#print(resultados)
grilla1 , grilla2 = np.meshgrid(puntos_prueba1, puntos_prueba2)
Z = np.zeros((len(grilla1), len(grilla1[0]) ))

#for i in range(len(grilla1)):
#    for j in range(len(grilla1[0])):
#        Z[i][j] = extraer(grilla1[i][j],grilla2[i][j],num_gap,tipo,resultados)      
    
lista = []
for i in range(len(puntos_prueba2)):
    for j in range(len(puntos_prueba1)):
        lista.append([grilla1[i][j], grilla2[i][j]])

print(lista)


[[1.5, 0.4], [2.5, 0.4], [3.5, 0.4], [1.5, 0.6], [2.5, 0.6], [3.5, 0.6], [1.5, 0.8], [2.5, 0.8], [3.5, 0.8]]


In [5]:
texto = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                         str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
            'vels' : { 'titulo' : lambda track,pola,identidad: 
                                         "Velocidades de grupo para " + str(track+1) + " bandas "
                                          + pola.upper() + " con " + identidad,
                         'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                         'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                     },
              '3d' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                         'ejey' : lambda _: 'Puntos de la rutina 2 (u.i)',
                         'ejez' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
           'color' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                         'ejey' : lambda _: 'Puntos de la rutina 2 (u.i)',
                         'ejez' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
           'proy1' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
           'proy2' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                         + ' en función de los puntos de la rutina', 
                         'ejex' : lambda _: 'Puntos de la rutina 2 (u.i)',
                         'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                     },
         'widgets' : {'puntos1' : lambda _: 'Puntos1',
                      'puntos2' : lambda _: 'Puntos2',
                      'num_gap' : lambda _: 'Gap',
                        'serie' : lambda _: 'Serie:'
                     }
        }


In [12]:
def traductor(param_red, resultados):
    c = 3.0e8
    factor = c/param_red

    for i in range(len(resultados['tefreqs'])):
        resultados['tefreqs'][i] = factor*resultados['tefreqs'][i]
    
    resultados['gaps']['Ancho TE'] = factor*resultados['gaps']['Ancho TE']
    resultados['gaps']['Prom TE'] = factor*resultados['gaps']['Prom TE']

    return resultados


def graficador(resultados,
               puntos1=0,
               puntos2=0,
               num_bandas=6,
               pola='te',
               texto=0,
               param_red=0.0,
               rango=0.0,
               mostrar_velocidades=False,
               mostrar_3d=False
              ):
    
    if isinstance(texto, int):
        texto = { 'bandas' : { 'titulo' : lambda track,pola,identidad: 
                                                 str(track+1) + " bandas " + pola.upper() + " con " + identidad,
                                 'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                                 'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                             },
                    'vels' : { 'titulo' : lambda track,pola,identidad: 
                                                 "Velocidades de grupo para " + str(track+1) + " bandas "
                                                  + pola.upper() + " con " + identidad,
                                 'ejex' : lambda _: 'Vector de onda (ka/2pi)',
                                 'ejey' : lambda _: 'Frecuencia (wa/2pic)'
                             },
                      '3d' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                                 + ' en función de los puntos de la rutina', 
                                 'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                                 'ejey' : lambda _: 'Puntos de la rutina 2 (u.i)',
                                 'ejez' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                             },
                   'color' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                                 + ' en función de los puntos de la rutina', 
                                 'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                                 'ejey' : lambda _: 'Puntos de la rutina 2 (u.i)',
                                 'ejez' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                             },
                   'proy1' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                                 + ' en función de los puntos de la rutina', 
                                 'ejex' : lambda _: 'Puntos de la rutina 1 (u.i)',
                                 'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                             },
                   'proy2' : { 'titulo' : lambda num_gap: 'del gap ' + str(num_gap) 
                                                 + ' en función de los puntos de la rutina', 
                                 'ejex' : lambda _: 'Puntos de la rutina 2 (u.i)',
                                 'ejey' : lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)'
                             },
                 'widgets' : {'puntos1' : lambda _: 'Puntos1',
                              'puntos2' : lambda _: 'Puntos2',
                              'num_gap' : lambda _: 'Gap',
                                'serie' : lambda _: 'Serie:'
                             }
                }

        
    
    try:
        with open('resultados-' + resultados + '.pkl', 'rb') as f:
            resultados = pickle.load(f)
    except:
        try:
            with open(resultados + '/resultados-' + resultados + '.pkl', 'rb') as f:
                resultados = pickle.load(f)
        except:
            pass
        
    if isinstance(puntos1, int): puntos1 = resultados['puntos'][0]
    if isinstance(puntos2, int): puntos2 = resultados['puntos'][1]
    
    grilla1 , grilla2 = np.meshgrid(puntos1, puntos2)
    Z = np.zeros((len(grilla1), len(grilla1[0]) )) 
    
    lista = []
    for i in range(len(puntos2)):
        for j in range(len(puntos1)):
            lista.append([grilla1[i][j], grilla2[i][j]])
    
    if (pola=='te'):
        llaves = ['tefreqs', 'tevels',  'Ancho TE', 'Promedio TE', 'Porcentaje TE']
    elif (pola=='tm'):
        llaves = ['tmfreqs', 'tmvels',  'Ancho TM', 'Promedio TM', 'Porcentaje TM']
    
    if (param_red != 0.0):
        resultados = traductor(param_red, resultados)
    
    def func_indice(valor,donde): 
        try:
            indice = int(np.where(donde == valor)[0][0])
        except:
            indice = donde.index(valor)
        return indice
    
    def graf_bandas(valor1, valor2, num_bandas, track=num_bandas-1):
        indice = func_indice([valor1,valor2],lista)
        fig, ax = plt.subplots()
        for i in range(track+1):
            k_vector = np.linspace(-0.5, 0.5, int(len(resultados[llaves[0]][indice][i])))
            ax.scatter(k_vector, resultados[llaves[0]][indice][i], c='r', s=0.1)
        
        parametros = resultados['parametros'][indice]
        identidad = str(parametros[0])
        for i in range(len(parametros)-1):
            identidad = identidad + '-' + str(parametros[i+1])

        ax.set_xlabel(texto['bandas']['ejex'](0))
        ax.set_ylabel(texto['bandas']['ejey'](0))
        ax.set_title(texto['bandas']['titulo'](track,pola,identidad))
        
        if (rango != 0.0):
            ax.set_ylim(rango)
        
    def graf_vels(valor1, valor2, num_bandas=num_bandas, track=num_bandas-1):
        indice = func_indice([valor1,valor2],lista)
        fig, ax = plt.subplots()
        for i in range(track+1):
            k_vector = np.linspace(-0.5, 0.5, int(len(resultados[llaves[1]][indice][i])))
            ax.scatter(k_vector, resultados[llaves[1]][indice][i], c='r', s=0.1)
        
        parametros = resultados['parametros'][indice]
        identidad = str(parametros[0])
        for i in range(len(parametros)-1):
            identidad = identidad + '-' + str(parametros[i+1])

        ax.set_xlabel(texto['bandas']['ejex'](0))
        ax.set_ylabel(texto['bandas']['ejey'](0))
        ax.set_title(texto['bandas']['titulo'](track,pola,identidad))
        
        if (rango != 0.0):
            ax.set_ylim(rango)
            
    def graf_3d(num_gap=1, tipo=llaves[2]):
        #grilla1 , grilla2 = np.meshgrid(puntos1, puntos2)
        #Z = np.zeros((len(grilla1), len(grilla1[0]) ))
        for i in range(len(grilla1)):
            for j in range(len(grilla1[0])):
                indice = func_indice([grilla1[i][j],grilla2[i][j]],lista)
                Z[i][j] = resultados['gaps'][tipo][num_gap-1][indice]
    
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
        #ax.plot_wireframe(grilla1,grilla2,Z)
        #ax.plot_surface(grilla1,grilla2,Z)
        ax.scatter(grilla1,grilla2,Z)
        ax.set_xlabel(texto['3d']['ejex'](0))
        ax.set_ylabel(texto['3d']['ejey'](0))
        ax.set_zlabel(texto['3d']['ejez'](0))
        if (tipo==llaves[2]):
            ax.set_title("Ancho " + texto['3d']['titulo'](num_gap))
        elif (tipo==llaves[3]):
            ax.set_title("Promedio " + texto['3d']['titulo'](num_gap))
        else:
            ax.set_title("Porcentaje " + texto['3d']['titulo'](num_gap))
        #ax.set_zlabel("omega")
        #ax.set_title("función de contraste y proporción de material")

    def graf_color(num_gap=1, tipo=llaves[2]):
        #grilla1 , grilla2 = np.meshgrid(puntos1, puntos2)
        #Z = np.zeros((len(grilla1), len(grilla1[0]) ))
        for i in range(len(grilla1)):
            for j in range(len(grilla1[0])):
                indice = func_indice([grilla1[i][j],grilla2[i][j]],lista)
                Z[i][j] = resultados['gaps'][tipo][num_gap-1][indice]     
    
        #fig = plt.figure()
        #plt.pcolormesh(puntos_epsilon_fino, puntos_tamanho_fino,Z)
        #plt.colorbar()
        
        fig, ax = plt.subplots()
        plt.pcolormesh(puntos1,puntos2,Z)
        cbar = plt.colorbar()
        ax.set_xlabel(texto['color']['ejex'](0))
        ax.set_ylabel(texto['color']['ejey'](0))
        cbar.set_label(texto['color']['ejez'](num_gap))
        if (tipo==llaves[2]):
            ax.set_title("Ancho " + texto['color']['titulo'](num_gap))
        elif (tipo==llaves[3]):
            ax.set_title("Promedio " + texto['color']['titulo'](num_gap))
        else:
            ax.set_title("Porcentaje " + texto['color']['titulo'](num_gap))
        
        #opcional
        #plt.xticks(puntos_epsilon_doble)
        #plt.yticks(puntos_tamanho_doble)
        
        
    def graf_proyeccion1(valor2, num_gap=1,tipo=llaves[2]):
        matriz = np.resize(resultados['gaps'][tipo][num_gap-1],( int(len(puntos2)),int(len(puntos1)) ))
        indice = func_indice(valor2,puntos2)
        fig, ax = plt.subplots()
        fig.subplots_adjust(0.2,0.2)
        plt.ticklabel_format(style='plain', useOffset=False)
        ax.scatter(puntos1, matriz[indice], c='r')
        ax.set_xlabel(texto['proy1']['ejex'](0))
        ax.set_ylabel(texto['proy1']['ejey'](num_gap))
        if (tipo==llaves[2]):
            ax.set_title("Ancho " + texto['proy1']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        elif (tipo==llaves[3]):
            ax.set_title("Promedio " + texto['proy1']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        else:
            ax.set_title("Porcentaje " + texto['proy1']['titulo'](num_gap))
        
    def graf_proyeccion2(valor1, num_gap=1,tipo=llaves[2]):
        matriz = np.resize(resultados['gaps'][tipo][num_gap-1],( int(len(puntos2)),int(len(puntos1)) ))
        indice = func_indice(valor1,puntos1)
        fig, ax = plt.subplots()
        fig.subplots_adjust(0.2,0.2)
        plt.ticklabel_format(style='plain', useOffset=False)
        ax.scatter(puntos2, np.transpose(matriz)[indice], c='r')
        ax.set_xlabel(texto['proy2']['ejex'](0))
        ax.set_ylabel(texto['proy2']['ejey'](num_gap))
        if (tipo==llaves[2]):
            ax.set_title("Ancho " + texto['proy2']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        elif (tipo==llaves[3]):
            ax.set_title("Promedio " + texto['proy2']['titulo'](num_gap))
            if (param_red != 0.0):
                ax.ticklabel_format(style='sci',scilimits=(0,0),axis='y')
        else:
            ax.set_title("Porcentaje " + texto['proy2']['titulo'](num_gap))
            
    
    ipywidgets.interact(graf_bandas,
                        valor1=ipywidgets.SelectionSlider(description=texto['widgets']['puntos1'](0),
                                                          options=puntos1
                                                         ),
                        valor2=ipywidgets.SelectionSlider(description=texto['widgets']['puntos2'](0),
                                                          options=puntos2
                                                         ),
                        num_bandas=fixed(num_bandas),
                        track=fixed(num_bandas-1)
                       )
    if (mostrar_velocidades==True):
        ipywidgets.interact(graf_vels,
                            valor1=ipywidgets.SelectionSlider(description=texto['widgets']['puntos1'](0),
                                                              options=puntos1
                                                             ),
                            valor2=ipywidgets.SelectionSlider(description=texto['widgets']['puntos2'](0),
                                                              options=puntos2
                                                             ),
                            num_bandas=fixed(num_bandas),
                            track=fixed(num_bandas-1)
                           )
    if (mostrar_3d==True):
        ipywidgets.interact(graf_3d,
                            num_gap=ipywidgets.SelectionSlider(description=texto['widgets']['num_gap'](0),
                                                               options=np.arange(1,num_bandas,1)
                                                              ),
                            tipo=ipywidgets.Dropdown(options=[llaves[2], llaves[3], llaves[4]],
                                                     value=llaves[2],
                                                     description=texto['widgets']['serie'](0),
                                                     disabled=False,
                                                    )
                           )
    
    ipywidgets.interact(graf_color,
                        num_gap=ipywidgets.SelectionSlider(description=texto['widgets']['num_gap'](0),
                                                         options=np.arange(1,num_bandas,1)
                                                        ),
                        tipo=ipywidgets.Dropdown(options=[llaves[2], llaves[3], llaves[4]],
                                             value=llaves[2],
                                             description=texto['widgets']['serie'](0),
                                             disabled=False,
                                            )
                       )
    
    ipywidgets.interact(graf_proyeccion1,
                        valor2=ipywidgets.SelectionSlider(description=texto['widgets']['puntos2'](0),
                                                         options=puntos2
                                                        ),
                        num_gap=ipywidgets.SelectionSlider(description=texto['widgets']['num_gap'](0),
                                                         options=np.arange(1,num_bandas,1)
                                                        ),
                        tipo=ipywidgets.Dropdown(options=[llaves[2], llaves[3], llaves[4]],
                                             value=llaves[2],
                                             description=texto['widgets']['serie'](0),
                                             disabled=False,
                                            )
                       )

    ipywidgets.interact(graf_proyeccion2,
                        valor1=ipywidgets.SelectionSlider(description=texto['widgets']['puntos1'](0),
                                                         options=puntos1
                                                        ),
                        num_gap=ipywidgets.SelectionSlider(description=texto['widgets']['num_gap'](0),
                                                         options=np.arange(1,num_bandas,1)
                                                        ),
                        tipo=ipywidgets.Dropdown(options=[llaves[2], llaves[3], llaves[4]],
                                             value=llaves[2],
                                             description=texto['widgets']['serie'](0),
                                             disabled=False,
                                            )
                       )
    return resultados

In [7]:
'''
graficador(puntos_prueba1,
           puntos_prueba2,
           resultados,
           num_bandas=6,
           pola='te',
           #ejex='Puntos de la rutina (u.i)',
           #ejey=lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)',
           #titulo=lambda num_gap: 'del gap ' + str(num_gap) + ' en función de los puntos de la rutina',
           #param_red=0.0,
           #rango=0.0,
          )
'''
resultados0['gaps']['Ancho TE'][0]

array([0.0583592 , 0.11719931, 0.14319887, 0.05075105, 0.08580411,
       0.09438936, 0.02776557, 0.04157666, 0.04313703])

In [8]:
#graficador(resultados0)#,
           #puntos_prueba1,
           #puntos_prueba2,
           #num_bandas=6,
           #pola='te',
           #ejex='Puntos de la rutina (u.i)',
           #ejey=lambda num_gap: 'Gap ' + str(num_gap) + ' (c/a)',
           #titulo=lambda num_gap: 'del gap ' + str(num_gap) + ' en función de los puntos de la rutina',
           #param_red=0.0,
           #rango=0.0,
          #)

In [14]:
ver = graficador('exp-doble-2mat', mostrar_velocidades=True, mostrar_3d=True, pola='te', rango=[0.0,3.0])


interactive(children=(SelectionSlider(description='Puntos1', options=(1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, …

interactive(children=(SelectionSlider(description='Puntos1', options=(1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, …

interactive(children=(SelectionSlider(description='Gap', options=(1, 2, 3, 4, 5), value=1), Dropdown(descripti…

interactive(children=(SelectionSlider(description='Gap', options=(1, 2, 3, 4, 5), value=1), Dropdown(descripti…

interactive(children=(SelectionSlider(description='Puntos2', options=(0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, …

interactive(children=(SelectionSlider(description='Puntos1', options=(1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5, …

In [18]:
print(ver['gaps']['Ancho TE'])


[[0.         0.         0.         ... 0.01971887 0.01929016 0.01884936]
 [0.00199601 0.00199601 0.00199601 ... 0.03852835 0.03770405 0.0368524 ]
 [0.         0.         0.         ... 0.05563295 0.05448164 0.05328002]
 ...
 [0.         0.         0.         ... 0.05699205 0.07073288 0.08100035]
 [0.00199601 0.00199601 0.00199601 ... 0.04008898 0.05632264 0.06900159]
 [0.         0.         0.         ... 0.02144044 0.03978587 0.05479635]]
